<a href="https://colab.research.google.com/github/jwkanggist/EverybodyTensorflow2.0/blob/master/lab19_basic_rnn_tf1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# LAB19 Basic RNN using Tensorflow

- 간단한 seq to seq RNN의 기본동작 확인하기
-  
- Tanh activation
- TWO RNN  CELL → Single state RNN 
- 5 recurrent neuron 
- Each Xt  is 3 x 1 vector
- Each Yt is 5 x 1 vector


In [1]:
# preprocessor parts

from __future__ import absolute_import, division, print_function, unicode_literals

# Install TensorFlow
try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass

import tensorflow.compat.v1 as tf

import numpy as np

from matplotlib import cm
from matplotlib import gridspec
import matplotlib.pyplot as plt

from datetime import datetime

tf.disable_eager_execution()

TensorFlow 2.x selected.


In [2]:
  # for Tensorboard use
  LOG_DIR = 'drive/data/tb_logs'
	
  !wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
  !unzip ngrok-stable-linux-amd64.zip
	
  import os
  if not os.path.exists(LOG_DIR):
    os.makedirs(LOG_DIR)
	  
  get_ipython().system_raw(
      'tensorboard --logdir {} --host 0.0.0.0 --port 6006 &'
      .format(LOG_DIR))
	
  get_ipython().system_raw('./ngrok http 6006 &')
	
  !curl -s http://localhost:4040/api/tunnels | python3 -c \
      "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

--2020-01-18 15:23:32--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 34.235.253.190, 52.72.59.23, 52.4.202.19, ...
Connecting to bin.equinox.io (bin.equinox.io)|34.235.253.190|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13773305 (13M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip’

ngrok-stable-linux- 100%[===================>]  13.13M  67.6MB/s    in 0.2s    

2020-01-18 15:23:32 (67.6 MB/s) - ‘ngrok-stable-linux-amd64.zip’ saved [13773305/13773305]

Archive:  ngrok-stable-linux-amd64.zip
  inflating: ngrok                   
http://343111b2.ngrok.io


In [0]:
# test sequential data
X0_batch = np.array([[0,1,2],
                      [3,4,5],
                      [6,7,8],
                      [9,0,1],
                      ])
X1_batch = np.array([[9,8,7],
                      [6,5,4],
                      [3,2,1],
                      [0,9,8]])

In [0]:
model_config = \
{
    'batch_size': None,
    'n_input'   : 3,
    'n_output'  : 5,
    'num_steps' : 2,
    'dtype'     : tf.float32
}

weigth_initializer = tf.initializers.he_uniform()


def get_rnn_model(X0,X1,scope):
    Wx_shape = [model_config['n_input'],
                model_config['n_output']]

    Wh_shape = [model_config['n_output'],
                model_config['n_output']]

    bias_shape    = [1,model_config['n_output']]

    with tf.name_scope(name=scope,values=[X0,X1]):

        Wx = tf.get_variable(name='weight_x',
                             shape=Wx_shape,
                             initializer=weigth_initializer)

        Wh = tf.get_variable(name='weight_h',
                             shape=Wh_shape,
                             initializer=weigth_initializer)
        b  = tf.get_variable(name='bias',
                             shape=bias_shape,
                             initializer=weigth_initializer)

        Y0 = tf.tanh(tf.matmul(X0,Wx) + b)
        Y1 = tf.tanh(tf.matmul(X1,Wx) + tf.matmul(Y0,Wh) + b)

    return Y0,Y1



In [6]:

if __name__ == '__main__':

    input_shape     = [model_config['batch_size'],
                       model_config['n_input']]

    output_shape    = [model_config['batch_size'],
                       model_config['n_output']]

    X0 = tf.placeholder(dtype = model_config['dtype'],
                        shape = input_shape,
                        name  = 'X0')

    X1 = tf.placeholder(dtype= model_config['dtype'],
                        shape= input_shape,
                        name = 'X1')

    scope = 'basic_rnn_matmul_model'
    Y0,Y1 = get_rnn_model(X0,X1,scope)

    summary_writer = tf.summary.FileWriter(logdir=LOG_DIR)
    summary_writer.add_graph(graph=tf.get_default_graph())
    init = tf.global_variables_initializer()

    X0_batch = np.array([[0,1,2],
                         [3,4,5],
                         [6,7,8],
                         [9,0,1],
                         ])
    X1_batch = np.array([[9,8,7],
                         [6,5,4],
                         [3,2,1],
                         [0,9,8]])

    with tf.Session() as sess:

        sess.run(init)
        Y0_val,Y1_val = sess.run(fetches=[Y0,Y1],feed_dict={X0:X0_batch,
                                                            X1:X1_batch})

    print('Y0_val = %s' % Y0_val)
    print('Y1_val = %s' % Y1_val)
    summary_writer.close()

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Y0_val = [[ 9.9380141e-01 -8.1916928e-01 -7.8262842e-01 -9.9899250e-01
  -8.0430484e-04]
 [ 1.0000000e+00  8.7570202e-01  4.9729702e-01 -9.9999869e-01
   5.8231091e-01]
 [ 1.0000000e+00  9.9912745e-01  9.7288239e-01 -1.0000000e+00
   8.6990982e-01]
 [ 9.9997592e-01  9.9982184e-01  9.9999756e-01 -9.8241669e-01
   9.9993986e-01]]
Y1_val = [[ 1.          0.9999995   0.999813   -1.          0.9999591 ]
 [ 1.          0.99990845  0.99989486 -0.9999977   0.9980418 ]
 [ 0.99934065  0.9759021   0.9984516  -0.9984355   0.97296387]
 [ 1.          0.87395024 -0.77864987 -1.         -0.95521426]]
